Fuzzy Methode

In [ ]:
# Install  Packets

import sys
!{sys.executable} -m pip install lightfm
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install surprise
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikit-surprise 
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install google.colab
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install fastparquet
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install pylance

#Only Once

In [6]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# TO Fuzz
from fuzzywuzzy import fuzz

# 
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

In [2]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
movie_tile_File_new = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Titles_clear.gzip'
combined_data_1 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_1.txt'
combined_data_2 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_2.txt'
combined_data_3 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_3.txt'
combined_data_4 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_4.txt'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Combined_Data_All.gzip'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'


# Read Parquet
data = pd.read_parquet(new_Combined)
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32', 'Movie_Id': 'Int64'}
data= data.astype(Convert_dic_data)

# Reorder Columns
del new_Combined, Convert_dic_data
data = data[['Cust_Id', 'Movie_Id', 'Rating']]
## Read Movie Data
movie_titles = pd.read_csv(movie_tile_File,
                           encoding = "ISO-8859-1",
                           delimiter= '\t',
                           header = None,
                           names = ['Target'])
                           # Speicher alle Daten erst in eine Reihe, danach trennt er diese
movie_titles[['Movie_Id', 'Year', 'Name']] = movie_titles['Target'].str.split(pat=",",n=2, expand=True)   
movie_titles= movie_titles.drop(['Target', 'Year'], axis= 1)  
Convert_dic= {'Movie_Id': 'int64', 'Name': 'str'}
movie_titles = movie_titles.astype(Convert_dic)
#movie_titles.astype({'Movie_Id': 'int64', 'Name': 'str'}).dtypes
#movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
#movie_titles = movie_titles.reset_index(drop=True)
display(movie_titles)
del movie_tile_File, Convert_dic 


,Movie_Id,Name
0,1,Dinosaur Planet
1,2,Isle of Man TT 2004 Review
2,3,Character
3,4,Paula Abdul's Get Up & Dance
4,5,The Rise and Fall of ECW
...,...,...
17765,17766,Where the Wild Things Are and Other Maurice Se...
17766,17767,Fidel Castro: American Experience
17767,17768,Epoch
17768,17769,The Company


In [3]:
df = data
# Filter sparse movies
min_movie_ratings = 1000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings, data

#Reorder the Columns
df_filterd = df_filterd[['Cust_Id', 'Movie_Id', 'Rating']]

print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))


Shape User-Ratings unfiltered:	(100480507, 3)
Shape User-Ratings filtered:	(75010415, 3)


In [4]:
# create mapper from movie title to index
# pivot and create movie-user matrix
movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

# create mapper from movie title to index
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}
# transform matrix to scipy sparse matrix
# movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [5]:
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=120, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=120)

In [ ]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    #https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
    #https://www.kaggle.com/code/razor08/knn-based-collaborative-filtering?scriptVersionId=81850169  Wo kommmt der Code her
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_movie)
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [ ]:
my_favorite = 'Haiku Tunnel'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=5)

In [8]:
# The Reader class is used to parse a file containing ratings.
# The file is assumed to specify only one rating per line, such as in the df_ratings_temp file above.
reader = Reader()
ratings_by_users = Dataset.load_from_df(df_filterd[['Cust_Id', 'Movie_Id','Rating']], reader)

Könnte interessant sein
https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/surprise_svd_deep_dive.ipynb
https://www.relataly.com/building-a-movie-recommender-using-collaborative-filtering/4376/#h-step-3-split-the-data-in-train-and-test

In [9]:
# Split the Data into train and test
train_df, test_df = train_test_split(ratings_by_users, test_size=.4)
#algo = BaselineOnly(bsl_options)
#predictions = algo.fit(trainset).test(testset)
#accuracy.rmse(predictions)

In [10]:
# train an SVD model
svd_model = SVD()
svd_model_trained = svd_model.fit(train_df)

In [11]:
# 10-fold cross validation 
cross_val_results = cross_validate(svd_model_trained, ratings_by_users, measures=['RMSE', 'MAE', 'MSE'], cv=10, verbose=False)
test_mae = cross_val_results['test_mae']

# mean squared errors per fold
df_test_mae = pd.DataFrame(test_mae, columns=['Mean Absolute Error'])
df_test_mae.index = np.arange(1, len(df_test_mae) + 1)
df_test_mae.sort_values(by='Mean Absolute Error', ascending=False).head(15)

# plot an overview of the performance per fold
plt.figure(figsize=(6,4))
sns.set_theme(style="whitegrid")
sns.barplot(y='Mean Absolute Error', x=df_test_mae.index, data=df_test_mae, color="b")
# plt.title('Mean Absolute Error')

MemoryError: 

In [ ]:
# predict ratings for a single user_id and for all movies
user_id = 400 # some test user from the ratings file

# create the predictions
pred_series= []
df_ratings_filtered = df_filterd[df_filterd['Cust_Id'] == user_id]

print(f'number of ratings: {df_ratings_filtered.shape[0]}')
for movie_id, name in zip(movie_titles.index, movie_titles['Name']):
    # check if the user has already rated a specific movie from the list
    rating_real = df_filterd.query(f'Movie_Id == {movie_id}')['Rating'].values[0] if movie_id in df_ratings_filtered['Movie_Id'].values else 0
    # generate the prediction
    rating_pred = svd_model_trained.predict(user_id, movie_id, rating_real, verbose=False)
    # add the prediction to the list of predictions
    pred_series.append([movie_id, name, rating_pred.est, rating_real])

# print the results
df_recommendations = pd.DataFrame(pred_series, columns=['Movie_Id', 'Name', 'predicted_rating', 'actual_rating'])
df_recommendations.sort_values(by='predicted_rating', ascending=False).head(15)

In [ ]:
# predict ratings for the combination of user_id and movie_id
user_id = 217 # some test user from the ratings file
movie_id = 4002
rating_real = df_filterd.query(f'Movie_Id == {movie_id} & Cust_Id == {user_id}')['Rating'].values[0]
movie_title = movie_titles[movie_titles.index == 862]['Name'].values[0]

print(f'Movie title: {movie_title}')
print(f'Actual rating: {rating_real}')

# predict and show the result
rating_pred = svd_model_trained.predict(user_id, movie_id, rating_real, verbose=True)

Andere Methode

In [ ]:
train, test = python_random_split(df_filterd, 0.75)